In [1]:
import numpy as np
import openmc
import openmc.deplete
from pyne import serpent
import re
from openmc.data import zam, gnd_name
from openmc.deplete.coupled_operator import _get_nuclides_with_data

In [8]:
og_mats = openmc.Materials.from_xml()
fuel = og_mats[0]

In [9]:
# Load OpenMC depletion results
dep_omc = openmc.deplete.Results(filename='openmc-results/depletion_results.h5')

In [12]:
name_dict = dep_omc[0].index_nuc
sorted_names = sorted(name_dict.items(), key=lambda x:x[1])
mat = dep_omc.export_to_materials(0)
serpent_mats = []
tot_mass = mat[0].get_mass()
serp_spec = []
og_nucs = fuel.get_nuclides()
nuclides_with_data = _get_nuclides_with_data('/home/ooblack/projects/cross-section-libraries/endfb71_h5/cross_sections.xml')
for name, idx in sorted_names:
    mass_percent = mat[0].get_mass(name) / tot_mass
    z, a, m = zam(name)
    serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
    if name in nuclides_with_data:
        serp_zam = ''.join([serp_zam, '.82c'])
    else:
        serp_zam = ''.join([serp_zam, str(m)])
    serp_space = ''.join([" "]*(20 - len(serp_zam)))
    serp_zam = serp_space + serp_zam
    if name not in og_nucs:
        mass_percent = 0.0
    serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')
    
    

In [13]:
with open('serp_spec.txt', 'w') as f:
    f.writelines(serp_spec)